# Visualizing national teams during the European Championship 2020

To get a better understand of the home clubs of each player and -- most interestingly -- the composition of the national teams, I scraped data and used it for visualization. I describe my steps below.

To replicate the steps, you need the following packages:

In [4]:
import pandas as pd
from pandas import DataFrame
from bs4 import BeautifulSoup
import requests
import re
import ipywidgets as widgets
from ipywidgets import interactive, interact
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objects as go
import plotly
import chart_studio.plotly as py

I rely on data by [The Athletic](https://theathletic.com/2594478/2021/06/15/euro-2020-squads-teams-list/).
To scrape the data, I followed this [guide](https://medium.com/@devkosal/scraping-data-with-beautifulsoup-and-selectorgadget-in-python-3-decf798e1a1e).
It is a combination of [SelectorGadget](https://selectorgadget.com) and [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/). BeautifulSoup pulls data from HTML and XML files. SelectorGadget helps to define the CSS selectors to extract specific parts of BeautifulSoup's output.
You will get an HTML output that I restrict to the desired text (using SelectorGadget to define which text I need). Since the resulting object still contains output that I did not want to have included, I further had to do some data wrangling until I got the desired data frame.

## Scrape the data

In [5]:
# Load URL in beautiful soup
url = 'https://theathletic.com/2594478/2021/06/15/euro-2020-squads-teams-list/'

# Get the data
res = requests.get(url)

# Use beautiful soup to parse it
soup = BeautifulSoup(res.text, "html.parser")

# This is how the output looks like -- not really beautiful, right?
soup.head()

[<title>Euro 2020: Every national team’s squad list for this summer’s European Championship – The Athletic</title>,
 <meta charset="utf-8"/>,
 <script type="text/javascript">(window.NREUM||(NREUM={})).loader_config={xpid:"VQUEVVZVDRABU1FRAQkAVQ==",licenseKey:"e5d6a2e943",applicationID:"17036873"};window.NREUM||(NREUM={}),__nr_require=function(t,e,n){function r(n){if(!e[n]){var o=e[n]={exports:{}};t[n][0].call(o.exports,function(e){var o=t[n][1][e];return r(o||e)},o,o.exports)}return e[n].exports}if("function"==typeof __nr_require)return __nr_require;for(var o=0;o<n.length;o++)r(n[o]);return r}({1:[function(t,e,n){function r(t){try{c.console&&console.log(t)}catch(e){}}var o,i=t("ee"),a=t(29),c={};try{o=localStorage.getItem("__nr_flags").split(","),console&&"function"==typeof console.log&&(c.console=!0,o.indexOf("dev")!==-1&&(c.dev=!0),o.indexOf("nr_dev")!==-1&&(c.nrDev=!0))}catch(s){}c.nrDev&&i.on("internal-error",function(t){r(t.stack)}),c.dev&&i.on("fn-err",function(t,e,n){r(n.stack)}

In [6]:
# SelectorGadget to the rescue :-)
# I extract the required content (I determined the selection criteria "p , h3" using the SelectorGadget)
plain_text = soup.select("p , h3")

# Extract the plain text (it turned out to work best for me)
text = soup.get_text()

# It's now way better:
text

"\n\n\nEuro 2020: Every national team’s squad list for this summer’s European Championship – The Athletic\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n          Press 'Enter' to see all results        \n\n\n\n\n\n\n\n\n\n\n\n\n\n\nMost Recent Stories\nAuthors\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nLog In\nSubscribe\n\n\n\n\n\n\n\n\n\nReal Time\nCities\nNHL\nMLB\nNFL\nNBA\nCFB\nCBB\nSoccer\nBetting\n• • •\n\n\n\n\n\n\n\n\n\n\n\n\n                  Arizona                \n\n                  Atlanta                \n\n                  Baltimore                \n\n                  Bay Area                \n\n                  Boston                \n\n                  Buffalo                \n\n                  Calgary                \n\n                  Carolina                \n\n\n\n                  Chicago                \n\n                  Cincinnati      

In [7]:
# Split the text string (this will lead to separate string on which we can filter)
text_split = text.split("\n\n\n\n\n")

text_split

['\n\n\nEuro 2020: Every national team’s squad list for this summer’s European Championship – The Athletic',
 '',
 '',
 '',
 '\n\n ',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 "\n\n          Press 'Enter' to see all results        ",
 '',
 '',
 'Most Recent Stories\nAuthors',
 '',
 '',
 '',
 '\n\n\n\nLog In\nSubscribe',
 '',
 'Real Time\nCities\nNHL\nMLB\nNFL\nNBA\nCFB\nCBB\nSoccer\nBetting\n• • •',
 '',
 '\n\n\n                  Arizona                \n\n                  Atlanta                \n\n                  Baltimore                \n\n                  Bay Area                \n\n                  Boston                \n\n                  Buffalo                \n\n                  Calgary                \n\n                  Carolina                \n\n\n\n                  Chicago                \n\n                  Cincinnati                \n\n                  Cleveland                \n\n                  Columbus                \n\n                  Dall

In [8]:
# Using index, I select the part that is relevant for me:
text_split[119]

'Group A\nTurkey\nGoalkeepers: Altay Bayindir (Fenerbahce), Ugurcan Cakır (Trabzonspor), Mert Gunok (Istanbul Basaksehir)\nDefenders: Kaan Ayhan (Sassuolo), Zeki Celik (LOSC Lille), Merih Demiral (Juventus), Ozan Kabak (Liverpool), Umut Meras (Le Havre), Mert Muldur (Sassuolo), Caglar Soyuncu (Leicester), Ridvan Yilmaz (Besiktas)\nMidfielders: Taylan Antalyali (Galatasaray), Hakan Calhanoglu (AC Milan), Halil Dervisoglu (Brentford), Irfan Can Kahveci (Fenerbahce), Orkun Kokcu (Feyenoord), Abdulkadir Omur (Trabzonspor), Dorukhan Tokoz (Besiktas), Ozan Tufan (Fenerbahce), Okay Yokuslu (West Brom)\nForwards: Kerem Akturkoglu (Galatasaray), Kenan Karaman (Fortuna Düsseldorf), Enes Unal (Getafe), Cengiz Under (Leicester), Yusuf Yazici (LOSC Lille), Burak Yilmaz (LOSC Lille)\nItaly\nGoalkeepers: Gianluigi Donnarumma (Milan), Alex Meret (Napoli), Salvatore Sirigu (Torino)\nDefenders: Francesco Acerbi (Lazio), Alessandro Bastoni (Inter), Leonardo Bonucci (Juventus), Giorgio Chiellini (Juventus

In [9]:
# And I restrict my data to this part only
player = text_split[119]

And now it's all about data wrangling :-)

In [10]:
# Remove \xao (better, otherwise we have do deal with it later)
player = player.replace(u'\xa0', u' ')

# And I again split the data -- but this time based on \n
player_row = player.split("\n")

# And that's how it looks like now:
player_row

['Group A',
 'Turkey',
 'Goalkeepers: Altay Bayindir (Fenerbahce), Ugurcan Cakır (Trabzonspor), Mert Gunok (Istanbul Basaksehir)',
 'Defenders: Kaan Ayhan (Sassuolo), Zeki Celik (LOSC Lille), Merih Demiral (Juventus), Ozan Kabak (Liverpool), Umut Meras (Le Havre), Mert Muldur (Sassuolo), Caglar Soyuncu (Leicester), Ridvan Yilmaz (Besiktas)',
 'Midfielders: Taylan Antalyali (Galatasaray), Hakan Calhanoglu (AC Milan), Halil Dervisoglu (Brentford), Irfan Can Kahveci (Fenerbahce), Orkun Kokcu (Feyenoord), Abdulkadir Omur (Trabzonspor), Dorukhan Tokoz (Besiktas), Ozan Tufan (Fenerbahce), Okay Yokuslu (West Brom)',
 'Forwards: Kerem Akturkoglu (Galatasaray), Kenan Karaman (Fortuna Düsseldorf), Enes Unal (Getafe), Cengiz Under (Leicester), Yusuf Yazici (LOSC Lille), Burak Yilmaz (LOSC Lille)',
 'Italy',
 'Goalkeepers: Gianluigi Donnarumma (Milan), Alex Meret (Napoli), Salvatore Sirigu (Torino)',
 'Defenders: Francesco Acerbi (Lazio), Alessandro Bastoni (Inter), Leonardo Bonucci (Juventus), Gi

As you'll see, there are still some empty rows and rows that we do not necessarily need (for instance "Group A" etc.). We'll deal with this now:

In [11]:
# Replace "Group A" etc. with an empty string ("")
player_row = [s.replace("Group A", "") for s in player_row]
player_row = [s.replace("Group B", "") for s in player_row]
player_row = [s.replace("Group C", "") for s in player_row]
player_row = [s.replace("Group D", "") for s in player_row]
player_row = [s.replace("Group E", "") for s in player_row]
player_row = [s.replace("Group F", "") for s in player_row]
player_row = [s.replace("(Photo: Getty Images)", "") for s in player_row]
# I initially wanted to automatize it with a loop,
# but it did not work -- so I went back to a few copy-pasted lines of code.

Now that these rows turned into empty strings, we can remove them:

In [12]:
# Filter for non-empty strings
player_row = list(filter(None, player_row))

player_row

['Turkey',
 'Goalkeepers: Altay Bayindir (Fenerbahce), Ugurcan Cakır (Trabzonspor), Mert Gunok (Istanbul Basaksehir)',
 'Defenders: Kaan Ayhan (Sassuolo), Zeki Celik (LOSC Lille), Merih Demiral (Juventus), Ozan Kabak (Liverpool), Umut Meras (Le Havre), Mert Muldur (Sassuolo), Caglar Soyuncu (Leicester), Ridvan Yilmaz (Besiktas)',
 'Midfielders: Taylan Antalyali (Galatasaray), Hakan Calhanoglu (AC Milan), Halil Dervisoglu (Brentford), Irfan Can Kahveci (Fenerbahce), Orkun Kokcu (Feyenoord), Abdulkadir Omur (Trabzonspor), Dorukhan Tokoz (Besiktas), Ozan Tufan (Fenerbahce), Okay Yokuslu (West Brom)',
 'Forwards: Kerem Akturkoglu (Galatasaray), Kenan Karaman (Fortuna Düsseldorf), Enes Unal (Getafe), Cengiz Under (Leicester), Yusuf Yazici (LOSC Lille), Burak Yilmaz (LOSC Lille)',
 'Italy',
 'Goalkeepers: Gianluigi Donnarumma (Milan), Alex Meret (Napoli), Salvatore Sirigu (Torino)',
 'Defenders: Francesco Acerbi (Lazio), Alessandro Bastoni (Inter), Leonardo Bonucci (Juventus), Giorgio Chiell

The next step is a bit tricky but actually straightforward. I define an empty dictionary `teams` and loop over the list `player_row` to extract the national team identity (defined by `teams[player_row[i]]`) as well as all the information on the national teams (who the players are, what their home clubs are, and which position they play). The last part is extracted using `player_row[start:end]`.
If you look at the data above, you'll see that all the information is located in 5 rows (each comma shows a new row). I thus use the `range()` function and start counting in row `0`, over the length of the list `len(player_row)` and jump in `5` row steps.

In [13]:
# Define a dictionary
teams = {}
# Extract sub sets for each national team
# I use the "start, stop, step" arguments from range
for i in range(0, len(player_row), 5):
    start = i + 1
    end = i + 5
    teams[player_row[i]] = player_row[start:end]

In a next step, I turn it into a data frame.

In [14]:
# Turn it into a DF and do some wranglin
teams_df = DataFrame(teams)

teams_df.head()

,Turkey,Italy,Wales,Switzerland,Denmark,Finland,Belgium,Russia,Holland,Ukraine,...,Scotland,Czech Republic,Spain,Sweden,Poland,Slovakia,Hungary,Portugal,France,Germany
0,"Goalkeepers: Altay Bayindir (Fenerbahce), Ugur...","Goalkeepers: Gianluigi Donnarumma (Milan), Ale...","Goalkeepers: Adam Davies (Stoke City), Wayne H...","Goalkeepers: Yvon Mvogo (PSV), Yann Sommer (Mo...","Goalkeepers: Jonas Lossl (Midtjylland), Freder...","Goalkeepers: Lukas Hradecky (Leverkusen), Anss...","Goalkeepers: Thibaut Courtois (Real Madrid), S...","Goalkeepers: Yuri Dyupin (Rubin), Matvei Safon...","Goalkeepers: Marco Bizot (AZ), Tim Krul (Norwi...","Goalkeepers: Georgiy Bushchan (Dynamo Kyiv), A...",...,"Goalkeepers: Craig Gordon (Hearts), David Mars...","Goalkeepers: Ales Mandous (Olomouc), Jiri Pavl...","Goalkeepers: David de Gea (Manchester United),...","Goalkeepers: Karl-Johan Johnsson (Copenhagen),...","Goalkeepers: Lukasz Fabianski (West Ham), Wojc...","Goalkeepers: Martin Dubravka (Newcastle), Dusa...","Goalkeepers: Adam Bogdan (Ferencvaros), Dénes ...","Goalkeepers: Anthony Lopes (Lyon), Rui Patrici...","Goalkeepers: Hugo Lloris (Tottenham), Mike Mai...","Goalkeepers: Bernd Leno (Arsenal), Manuel Neue..."
1,"Defenders: Kaan Ayhan (Sassuolo), Zeki Celik (...","Defenders: Francesco Acerbi (Lazio), Alessandr...","Defenders: Ethan Ampadu (Chelsea), Ben Cabango...","Defenders: Manuel Akanji (Dortmund), Loris Ben...","Defenders: Joachim Andersen (Fulham), Nicolai ...","Defenders: Paulus Arajuuri (Pafos), Robert Iva...","Defenders: Toby Alderweireld (Tottenham), Dedr...","Defenders: Igor Diveev (CSKA Moskva), Georgi D...",Defenders: Patrick van Aanholt (Crystal Palace...,"Defenders: Eduard Sobol (Club Brugge), Illia Z...",...,"Defenders: Liam Cooper (Leeds), Declan Gallagh...","Defenders: Jan Boril (Slavia Praha), Jakub Bra...","Defenders: Jose Gaya (Valencia), Jordi Alba (B...",Defenders: Ludwig Augustinsson (Werder Bremen)...,"Defenders: Jan Bednarek (Southampton), Bartosz...","Defenders: David Hancko (Sparta Praha), Tomas ...","Defenders: Bendeguz Bolla (Fehervar), Endre Bo...","Defenders: Diogo Dalot (Manchester United), Ru...","Defenders: Lucas Digne (Everton), Leo Dubois (...",Defenders: Matthias Ginter (Borussia Monchengl...
2,"Midfielders: Taylan Antalyali (Galatasaray), H...","Midfielders: Nicolo Barella (Inter), Federico ...","Midfielders: Joe Allen (Stoke City), David Bro...","Midfielders: Christian Fassnacht (Young Boys),...","Midfielders: Anders Christiansen (Malmo), Thom...","Midfielders: Nikolai Alho (MTK Budapest), Fred...","Midfielders: Timothy Castagne (Leicester), Nac...",Midfielders: Dmitri Barinov (Lokomotiv Moskva)...,"Midfielders: Ryan Gravenberch (Ajax), Frenkie ...","Midfielders: Serhiy Sydorchuk (Dynamo Kyiv), R...",...,"Midfielders: Stuart Armstrong (Southampton), R...","Midfielders: Antonin Barak (Verona), Vladimir ...","Midfielders: Thiago Alcantara (Liverpool), Ser...","Midfielders: Jens-Lys Cajuste (Midtjylland), V...",Midfielders: Przemyslaw Frankowski (Chicago Fi...,"Midfielders: Laszlo Benes (Augsburg), Ondrej D...","Midfielders: Tamss Cseri (Mezokovesd), Daniel ...","Midfielders: William Carvalho (Real Betis), Da...","Midfielders: Kingsley Coman (Bayern), N’Golo K...","Midfielders: Emre Can (Borussia Dortmund), Ser..."
3,"Forwards: Kerem Akturkoglu (Galatasaray), Kena...","Forwards: Andrea Belotti (Torino), Domenico Be...","Forwards: Gareth Bale (Real Madrid), Kieffer M...","Forwards: Breel Embolo (Monchengladbach), Mari...","Forwards: Martin Braithwaite (Barcelona), Andr...","Forwards: Marcus Forss (Brentford), Lassi Lapp...","Forwards: Michy Batshuayi (Crystal Palace), Ch...","Forwards: Artem Dzyuba (Zenit), Aleksei Ionov ...","Forwards: Steven Berghuis (Feyenoord), Cody Ga...","Forwards: Roman Yaremchuk (Gent), Artem Besedi...",...,"Forwards: Che Adams (Southampton), Lyndon Dyke...","Forwards: Adam Hlozek (Sparta Praha), Michael ...","Forwards: Alvaro Morata (Juventus), Gerard Mor...","Forwards: Marcus Berg

The easiest way to deal with the data here, was to transpose it.

In [15]:
# Transpose DF
teams_t = teams_df.T

teams_t.head()

,0,1,2,3
Turkey,"Goalkeepers: Altay Bayindir (Fenerbahce), Ugur...","Defenders: Kaan Ayhan (Sassuolo), Zeki Celik (...","Midfielders: Taylan Antalyali (Galatasaray), H...","Forwards: Kerem Akturkoglu (Galatasaray), Kena..."
Italy,"Goalkeepers: Gianluigi Donnarumma (Milan), Ale...","Defenders: Francesco Acerbi (Lazio), Alessandr...","Midfielders: Nicolo Barella (Inter), Federico ...","Forwards: Andrea Belotti (Torino), Domenico Be..."
Wales,"Goalkeepers: Adam Davies (Stoke City), Wayne H...","Defenders: Ethan Ampadu (Chelsea), Ben Cabango...","Midfielders: Joe Allen (Stoke City), David Bro...","Forwards: Gareth Bale (Real Madrid), Kieffer M..."
Switzerland,"Goalkeepers: Yvon Mvogo (PSV), Yann Sommer (Mo...","Defenders: Manuel Akanji (Dortmund), Loris Ben...","Midfielders: Christian Fassnacht (Young Boys),...","Forwards: Breel Embolo (Monchengladbach), Mari..."
Denmark,"Goalkeepers: Jonas Lossl (Midtjylland), Freder...","Defenders: Joachim Andersen (Fulham), Nicolai ...","Midfielders: Anders Christiansen (Malmo), Thom...","Forwards: Martin Braithwaite (Barcelona), Andr..."


The next steps are about, moving the index column, renaming columns, and removing unneccessary strings.

In [16]:
# Move index to column
teams_t = teams_t.reset_index()
# Rename columns
teams_t.rename(columns={ teams_t.columns[0]: "national_team" }, inplace = True)
teams_t.rename(columns={ teams_t.columns[1]: "goalkeeper" }, inplace = True)
teams_t.rename(columns={ teams_t.columns[2]: "defender" }, inplace = True)
teams_t.rename(columns={ teams_t.columns[3]: "midfield" }, inplace = True)
teams_t.rename(columns={ teams_t.columns[4]: "forward" }, inplace = True)

# Remove strings from columns
teams_t["goalkeeper"] = [s.replace("Goalkeepers: ", "") for s in teams_t["goalkeeper"]]
teams_t["defender"] = [s.replace("Defenders: ", "") for s in teams_t["defender"]]
teams_t["midfield"] = [s.replace("Midfielders: ", "") for s in teams_t["midfield"]]
teams_t["forward"] = [s.replace("Forwards: ", "") for s in teams_t["forward"]]

teams_t.head()

,national_team,goalkeeper,defender,midfield,forward
0,Turkey,"Altay Bayindir (Fenerbahce), Ugurcan Cakır (Tr...","Kaan Ayhan (Sassuolo), Zeki Celik (LOSC Lille)...","Taylan Antalyali (Galatasaray), Hakan Calhanog...","Kerem Akturkoglu (Galatasaray), Kenan Karaman ..."
1,Italy,"Gianluigi Donnarumma (Milan), Alex Meret (Napo...","Francesco Acerbi (Lazio), Alessandro Bastoni (...","Nicolo Barella (Inter), Federico Bernardeschi ...","Andrea Belotti (Torino), Domenico Berardi (Sas..."
2,Wales,"Adam Davies (Stoke City), Wayne Hennessey (Cry...","Ethan Ampadu (Chelsea), Ben Cabango (Swansea C...","Joe Allen (Stoke City), David Brooks (Bournemo...","Gareth Bale (Real Madrid), Kieffer Moore (Card..."
3,Switzerland,"Yvon Mvogo (PSV), Yann Sommer (Monchengladbach...","Manuel Akanji (Dortmund), Loris Benito (Bordea...","Christian Fassnacht (Young Boys), Edimilson Fe...","Breel Embolo (Monchengladbach), Mario Gavranov..."
4,Denmark,"Jonas Lossl (Midtjylland), Frederik Ronnow (Sc...","Joachim Andersen (Fulham), Nicolai Boilesen (C...","Anders Christiansen (Malmo), Thomas Delaney (D...","Martin Braithwaite (Barcelona), Andreas Cornel..."


I still have all the names of player on each position in one column. I know R has a great package called [`splitstackshape`](https://www.r-project.org/nosvn/pandoc/splitstackshape.html) that automatically splits strings, based on a separation criteria in X columns. I couldn't find a Python equivalent, so I did it in a bit more tedious way.

In [17]:
# And now we can split each column based on ","
teams_t[['goalkeeper1', 'goalkeeper2', 'goalkeeper3']] = teams_t['goalkeeper'].str.split(pat= ',', expand=True)
teams_t[['defender1', 'defender2', 'defender3', 'defender4', 'defender5', 'defender6', 'defender7', 'defender8', 'defender9', 'defender10']] = teams_t['defender'].str.split(pat= ',', expand=True)
teams_t[['midfield1', 'midfield2', 'midfield3', 'midfield4', 'midfield5', 'midfield6', 'midfield7', 'midfield8', 'midfield9', 'midfield10', 'midfield11', 'midfield12']] = teams_t['midfield'].str.split(pat= ',', expand=True)
teams_t[['forward1', 'forward2', 'forward3', 'forward4', 'forward5', 'forward6', 'forward7', 'forward8']] = teams_t['forward'].str.split(pat= ',', expand=True)

teams_t.head()

,national_team,goalkeeper,defender,midfield,forward,goalkeeper1,goalkeeper2,goalkeeper3,defender1,defender2,...,midfield11,midfield12,forward1,forward2,forward3,forward4,forward5,forward6,forward7,forward8
0,Turkey,"Altay Bayindir (Fenerbahce), Ugurcan Cakır (Tr...","Kaan Ayhan (Sassuolo), Zeki Celik (LOSC Lille)...","Taylan Antalyali (Galatasaray), Hakan Calhanog...","Kerem Akturkoglu (Galatasaray), Kenan Karaman ...",Altay Bayindir (Fenerbahce),Ugurcan Cakır (Trabzonspor),Mert Gunok (Istanbul Basaksehir),Kaan Ayhan (Sassuolo),Zeki Celik (LOSC Lille),...,None,None,Kerem Akturkoglu (Galatasaray),Kenan Karaman (Fortuna Düsseldorf),Enes Unal (Getafe),Cengiz Under (Leicester),Yusuf Yazici (LOSC Lille),Burak Yilmaz (LOSC Lille),None,None
1,Italy,"Gianluigi Donnarumma (Milan), Alex Meret (Napo...","Francesco Acerbi (Lazio), Alessandro Bastoni (...","Nicolo Barella (Inter), Federico Bernardeschi ...","Andrea Belotti (Torino), Domenico Berardi (Sas...",Gianluigi Donnarumma (Milan),Alex Meret (Napoli),Salvatore Sirigu (Torino),Francesco Acerbi (Lazio),Alessandro Bastoni (Inter),...,None,None,Andrea Belotti (Torino),Domenico Berardi (Sassuolo),Ciro Immobile (Lazio),Lorenzo Insigne (Napoli),Giacomo Raspadori (Sassuolo),None,None,None
2,Wales,"Adam Davies (Stoke City), Wayne Hennessey (Cry...","Ethan Ampadu (Chelsea), Ben Cabango (Swansea C...","Joe Allen (Stoke City), David Brooks (Bournemo...","Gareth Bale (Real Madrid), Kieffer Moore (Card...",Adam Davies (Stoke City),Wayne Hennessey (Crystal Palace),Danny Ward (Leicester City),Ethan Ampadu (Chelsea),Ben Cabango (Swansea City),...,None,None,Gareth Bale (Real Madrid),Kieffer Moore (Cardiff City),Tyler Roberts (Leeds United),None,None,None,None,None
3,Switzerland,"Yvon Mvogo (PSV), Yann Sommer (Monchengladbach...","Manuel Akanji (Dortmund), Loris Benito (Bordea...","Christian Fassnacht (Young Boys), Edimilson Fe...","Breel Embolo (Monchengladbach), Mario Gavranov...",Yvon Mvogo (PSV),Yann Sommer (Monchengladbach),Jonas Omlin (Montpellier),Manuel Akanji (Dortmund),Loris Benito (Bordeaux),...,None,None,Breel Embolo (Monchengladbach),Mario Gavranovic (Dinamo Zagreb),Haris Seferovic (Benfica),None,None,None,None,None
4,Denmark,"Jonas Lossl (Midtjylland), Frederik Ronnow (Sc...","Joachim Andersen (Fulham), Nicolai Boilesen (C...","Anders Christiansen (Malmo), Thomas Delaney (D...","Martin Braithwaite (Barcelona), Andreas Cornel...",Jonas Lossl (Midtjylland),Frederik Ronnow (Schalke),Kasper Schmeichel (Leicester),Joachim Andersen (Fulham),Nicolai Boilesen (Copenhagen),...,None,None,Martin Braithwaite (Barcelona),Andreas Cornelius (Parma),Mikkel Damsgaard (Sampdoria),Kasper Dolberg (Nice),Andreas Skov Olsen (Bologna),Yussuf Poulsen (Leipzig),Jonas Wind (Copenhagen),None


Not exactly what I want, so I still have to do some cleaning :-)
To get the format, I am looking for, I use `pd.melt`.

In [18]:
# Now transform columns into rows and keep the content
teams_m = pd.melt(teams_t, id_vars=['national_team'],
        value_vars=['goalkeeper1', 'goalkeeper2', 'goalkeeper3', 'defender1', 'defender2', 'defender3', 'defender4', 'defender5', 'defender6', 'defender7', 'defender8', 'defender9', 'defender10',
                    'midfield1', 'midfield2', 'midfield3', 'midfield4', 'midfield5', 'midfield6', 'midfield7', 'midfield8', 'midfield9', 'midfield10', 'midfield11', 'midfield12',
                    'forward1', 'forward2', 'forward3', 'forward4', 'forward5', 'forward6', 'forward7', 'forward8'], var_name = "position", value_name = "player")

teams_m.head()

,national_team,position,player
0,Turkey,goalkeeper1,Altay Bayindir (Fenerbahce)
1,Italy,goalkeeper1,Gianluigi Donnarumma (Milan)
2,Wales,goalkeeper1,Adam Davies (Stoke City)
3,Switzerland,goalkeeper1,Yvon Mvogo (PSV)
4,Denmark,goalkeeper1,Jonas Lossl (Midtjylland)


That looks almost like I want it to look like! I just want to remove the numbers in the position and empty rows (with no player) as well.

In [19]:
# Remove numbers from "position" column
teams_m['position'] = teams_m['position'].str.replace('\d+', '')

# Remove also empty rows with no value in "player"
teams_m = teams_m[teams_m['player'].notnull()]

teams_m.head()

<ipython-input-19-91fe747bfe4c>:2: FutureWarning:

The default value of regex will change from True to False in a future version.



,national_team,position,player
0,Turkey,goalkeeper,Altay Bayindir (Fenerbahce)
1,Italy,goalkeeper,Gianluigi Donnarumma (Milan)
2,Wales,goalkeeper,Adam Davies (Stoke City)
3,Switzerland,goalkeeper,Yvon Mvogo (PSV)
4,Denmark,goalkeeper,Jonas Lossl (Midtjylland)


Well, almost there :-) Now I need to split the last string ("player") into "player_name" and "home_club".

In [20]:
teams_m['home_club'] = teams_m.player.str.extract(r"\((.*?)\)", expand=True)
teams_m['player_name'] = teams_m.player.str.replace(r"\(.*\)","")

# And, for the visualization, I also add a "count"
teams_m["count"] = 1

teams_m.head()

<ipython-input-20-61f2417f78a4>:2: FutureWarning:

The default value of regex will change from True to False in a future version.



,national_team,position,player,home_club,player_name,count
0,Turkey,goalkeeper,Altay Bayindir (Fenerbahce),Fenerbahce,Altay Bayindir,1
1,Italy,goalkeeper,Gianluigi Donnarumma (Milan),Milan,Gianluigi Donnarumma,1
2,Wales,goalkeeper,Adam Davies (Stoke City),Stoke City,Adam Davies,1
3,Switzerland,goalkeeper,Yvon Mvogo (PSV),PSV,Yvon Mvogo,1
4,Denmark,goalkeeper,Jonas Lossl (Midtjylland),Midtjylland,Jonas Lossl,1


# Visualization

And now I can visualize the plots :-) I follow [this approach](https://medium.com/kenlok/how-to-create-sankey-diagrams-from-dataframes-in-python-e221c1b4d6b0) which also gives the great function `genSankey` below.

In [21]:
def genSankey(df, cat_cols=[], value_cols='', title='Sankey Diagram'):
    # maximum of 6 value cols -> 6 colors
    colorPalette = ['#4B8BBE', '#306998', '#FFE873', '#FFD43B', '#646464']
    labelList = []
    colorNumList = []
    for catCol in cat_cols:
        labelListTemp = list(set(df[catCol].values))
        colorNumList.append(len(labelListTemp))
        labelList = labelList + labelListTemp

    # remove duplicates from labelList
    labelList = list(dict.fromkeys(labelList))

    # define colors based on number of levels
    colorList = []
    for idx, colorNum in enumerate(colorNumList):
        colorList = colorList + [colorPalette[idx]] * colorNum

    # transform df into a source-target pair
    for i in range(len(cat_cols) - 1):
        if i == 0:
            sourceTargetDf = df[[cat_cols[i], cat_cols[i + 1], value_cols]]
            sourceTargetDf.columns = ['source', 'target', 'count']
        else:
            tempDf = df[[cat_cols[i], cat_cols[i + 1], value_cols]]
            tempDf.columns = ['source', 'target', 'count']
            sourceTargetDf = pd.concat([sourceTargetDf, tempDf])
        sourceTargetDf = sourceTargetDf.groupby(['source', 'target']).agg({'count': 'sum'}).reset_index()

    # add index for source-target pair
    sourceTargetDf['sourceID'] = sourceTargetDf['source'].apply(lambda x: labelList.index(x))
    sourceTargetDf['targetID'] = sourceTargetDf['target'].apply(lambda x: labelList.index(x))

    # creating the sankey diagram
    data = dict(
        type='sankey',
        node=dict(
            pad=15,
            thickness=20,
            line=dict(
                color="black",
                width=0.5
            ),
            label=labelList,
            color=colorList
        ),
        link=dict(
            source=sourceTargetDf['sourceID'],
            target=sourceTargetDf['targetID'],
            value=sourceTargetDf['count']
        )
    )

    layout = dict(
        title=title,
        font=dict(
            size=10
        )
    )

    fig = dict(data=[data], layout=layout)
    return fig

The function takes four arguments:
- `df` - the data frame
- `cat_cols` - the columns that you want to visualize in your Sankey diagram (for me it's `'player_name','home_club', 'national_team'`)
- `value_cols` - to define how thick the Sankeys are
- `title` - the title of your Sankey

I could simply plot the Sankey as it is using
`
fig = genSankey(german, cat_cols=['player_name','home_club', 'national_team'], value_cols='count', title='Composition of national teams')

ploty.offline.plot(sankey, validate = False)
`

but this would lead to an overload of Sankeys -- and it might be hard ot see anything. Python has also a way to make your results interactively accessible using [`ipywidgets`](https://ipywidgets.readthedocs.io/en/latest/index.html). I mainly follow [this guide](https://blog.ouseful.info/2016/12/29/simple-view-controls-for-pandas-dataframes-using-ipython-widgets/) to visualize my plot.

The code below produces an interactive plot where you can pick a national team and see which players are part of which club and how the clubs themselves contribute to the national team.

In [ ]:

items = sorted(teams_m['national_team'].unique().tolist())

def view(Team='', Club =''):
    df = teams_m[teams_m['national_team'] == Team]
    fig = genSankey(df,
                    cat_cols=['player_name','home_club', 'national_team'],
                    value_cols='count',
                    title='Composition of national teams')
    iplot(fig, validate=False)

w = widgets.Select(options=items)
interactive(view, Team=w)

interactive(children=(Select(description='Team', options=('Austria', 'Belgium', 'Croatia', 'Czech Republic', '…